In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
# cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("./data/tweet-sentiment-extraction/train.csv")
# above line will be different depending on where you saved your data, and your file name
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
df.sentiment.value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [5]:
# Remove URLs
import re
re.sub('https?://[A-Za-z0-9./]+','',df.text[0])

' I`d have responded, if I were going'